In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
import os
import random
from torch.autograd import Variable
import copy
from torch import nn, optim
import torch.optim as optim
import numpy as np
from torch.autograd import Variable
from collections import OrderedDict
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import csv
import time


In [2]:
def fix_seed(seed):
    # random
    random.seed(seed)
    # Numpy
    np.random.seed(seed)
    # Pytorch
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

SEED = 42
fix_seed(SEED)

In [3]:
class Argments():
  def __init__(self):
    self.batch_size = 20
    self.test_batch = 1000
    self.global_epochs = 300
    self.local_epochs = 2
    self.lr = None
    self.momentum = 0.9
    self.weight_decay = 10**-4.0
    self.clip = 20.0
    self.partience = 300
    self.worker_num = 20
    self.sample_num = 20
    self.cluster_num = 2
    self.device = device = torch.device('cuda:0'if torch.cuda.is_available() else'cpu')
    self.criterion = nn.CrossEntropyLoss()

args = Argments()

In [4]:
lr = 0

In [6]:
lr_list = []
lr_list.append(10**-3.0)
lr_list.append(10**-2.5)
lr_list.append(10**-2.0)
lr_list.append(10**-1.5)
lr_list.append(10**-1.0)
lr_list.append(10**-0.5)
lr_list.append(10**0.0)
lr_list.append(10**0.5)

args.lr = lr_list[lr]

In [9]:
class LocalDataset(torch.utils.data.Dataset):
  def __init__(self,dataset,worker_id):
    self.data = []
    self.target = []
    self.id = worker_id
    for data in dataset.take(len(dataset)):
      self.data.append(torch.tensor([data['pixels'].numpy()]))
      self.target.append(torch.tensor(data['label'].numpy().astype(np.int64)))

  def __getitem__(self, index):
    return self.data[index],self.target[index]

  def __len__(self):
    return len(self.data)

In [10]:
with open('../data/federated_trainset_femnist.pickle', 'rb') as f:
    all_federated_trainset = pickle.load(f)
with open('../data/federated_testset_femnist.pickle', 'rb') as f:
    all_federated_testset = pickle.load(f)
all_worker_num = len(all_federated_trainset)

In [11]:
worker_id_list = random.sample(range(all_worker_num),args.worker_num)
print(worker_id_list)
federated_trainset = []
federated_testset = []
for i in worker_id_list:
    federated_trainset.append(all_federated_trainset[i])
    federated_testset.append(all_federated_testset[i])

[2619, 456, 102, 3037, 1126, 1003, 914, 571, 3016, 419, 2771, 3033, 2233, 356, 2418, 1728, 130, 122, 383, 895]


In [12]:
federated_valset = [None]*args.worker_num
for i in range(args.worker_num):
  n_samples = len(federated_trainset[i])
  if n_samples==1:
    federated_valset[i] = copy.deepcopy(federated_trainset[i])
  else:
    train_size = int(len(federated_trainset[i]) * 0.7) 
    val_size = n_samples - train_size 
    federated_trainset[i],federated_valset[i] = torch.utils.data.random_split(federated_trainset[i], [train_size, val_size])

In [14]:
class CNN1(torch.nn.Module):
    def __init__(self):
        super(CNN1, self).__init__()
        self.conv2d_1 = torch.nn.Conv2d(1, 32, kernel_size=3)
        self.max_pooling = nn.MaxPool2d(2, stride=2)
        self.dropout_1 = nn.Dropout(0.25)
        self.flatten = nn.Flatten()
        self.linear_1 = nn.Linear(5408, 128)
        self.dropout_2 = nn.Dropout(0.5)
        self.linear_2 = nn.Linear(128,62)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.conv2d_1(x)
        x = self.max_pooling(x)
        x = self.dropout_1(x)
        x = self.flatten(x)
        x = self.relu(self.linear_1(x))
        x = self.dropout_2(x)
        x = self.linear_2(x)
        return x

class CNN2(torch.nn.Module):
    def __init__(self):
        super(CNN2, self).__init__()
        self.conv2d_1 = torch.nn.Conv2d(1, 32, kernel_size=3)
        self.max_pooling = nn.MaxPool2d(2, stride=2)
        self.conv2d_2 = torch.nn.Conv2d(32, 64, kernel_size=3)
        self.dropout_1 = nn.Dropout(0.25)
        self.flatten = nn.Flatten()
        self.linear_1 = nn.Linear(9216, 128)
        self.dropout_2 = nn.Dropout(0.5)
        self.linear_2 = nn.Linear(128,62)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.conv2d_1(x)
        x = self.conv2d_2(x)
        x = self.max_pooling(x)
        x = self.dropout_1(x)
        x = self.flatten(x)
        x = self.relu(self.linear_1(x))
        x = self.dropout_2(x)
        x = self.linear_2(x)
        return x

class CNN3(torch.nn.Module):
    def __init__(self):
        super(CNN3, self).__init__()
        self.conv2d_1 = torch.nn.Conv2d(1, 32, kernel_size=3)
        self.max_pooling = nn.MaxPool2d(2, stride=2)
        self.conv2d_2 = torch.nn.Conv2d(32, 64, kernel_size=3)
        self.conv2d_3 = torch.nn.Conv2d(64, 128, kernel_size=3)
        self.dropout_1 = nn.Dropout(0.25)
        self.flatten = nn.Flatten()
        self.linear_1 = nn.Linear(15488, 128)
        self.dropout_2 = nn.Dropout(0.5)
        self.linear_2 = nn.Linear(128,62)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.conv2d_1(x)
        x = self.conv2d_2(x)
        x = self.conv2d_3(x)
        x = self.max_pooling(x)
        x = self.dropout_1(x)
        x = self.flatten(x)
        x = self.relu(self.linear_1(x))
        x = self.dropout_2(x)
        x = self.linear_2(x)
        return x
    
class CNN4(torch.nn.Module):
    def __init__(self):
        super(CNN4, self).__init__()
        self.conv2d_1 = torch.nn.Conv2d(1, 32, kernel_size=3)
        self.max_pooling = nn.MaxPool2d(2, stride=2)
        self.conv2d_2 = torch.nn.Conv2d(32, 64, kernel_size=3)
        self.conv2d_3 = torch.nn.Conv2d(64, 128, kernel_size=3)
        self.conv2d_4 = torch.nn.Conv2d(128, 256, kernel_size=3)
        self.dropout_1 = nn.Dropout(0.25)
        self.flatten = nn.Flatten()
        self.linear_1 = nn.Linear(25600, 128)
        self.dropout_2 = nn.Dropout(0.5)
        self.linear_2 = nn.Linear(128,62)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.conv2d_1(x)
        x = self.conv2d_2(x)
        x = self.conv2d_3(x)
        x = self.conv2d_4(x)
        x = self.max_pooling(x)
        x = self.dropout_1(x)
        x = self.flatten(x)
        x = self.relu(self.linear_1(x))
        x = self.dropout_2(x)
        x = self.linear_2(x)
        return x

In [15]:
class Server():
  def __init__(self):
    self.models = []
    for i in range(args.cluster_num):
      self.models.append(CNN2())

  def model_initialize(self,workers):
    sample_worker = self.sample_worker(workers)
    self.send_models(sample_worker)
    for i,worker in enumerate(sample_worker):
      worker.cluster = i%args.cluster_num
      _ = worker.local_train()
    self.aggregate_models(sample_worker)

  def create_worker(self,federated_trainset,federated_valset,federated_testset):
    workers = []
    for i in range(args.worker_num):
      workers.append(Worker(federated_trainset[i],federated_valset[i],federated_testset[i]))
    return workers

  def sample_worker(self,workers):
    sample_worker = []
    sample_worker_num = random.sample(range(args.worker_num),args.sample_num)
    for i in sample_worker_num:
      sample_worker.append(workers[i])
    return sample_worker


  def send_models(self,workers):
    nums = 0
    for worker in workers:
      nums += worker.train_data_num

    for worker in workers:
      worker.aggregation_weight = 1.0*worker.train_data_num/nums
      worker.models = copy.deepcopy(self.models)
      for i in range(args.cluster_num):
        worker.models[i] = worker.models[i].to(args.device)

  def aggregate_models(self,workers):
    new_params = []
    for i in range(args.cluster_num):   
      new_params.append(OrderedDict())
    total_num = [0]*args.cluster_num
    for worker in workers:
      total_num[worker.cluster] += worker.train_data_num
    count = [0]*args.cluster_num    
    for worker in workers:
      worker_state = worker.models[worker.cluster].state_dict()
      for key in worker_state.keys():
        if count[worker.cluster]==0:
          new_params[worker.cluster][key] = 1.0*worker_state[key]*worker.train_data_num/total_num[worker.cluster]
        else:
          new_params[worker.cluster][key] += 1.0*worker_state[key]*worker.train_data_num/total_num[worker.cluster]
      count[worker.cluster] += 1
      for i in range(args.cluster_num):
        worker.models[i] = worker.models[i].to('cpu')
      del worker.models

    for i in range(args.cluster_num):
      if total_num[i]!=0:
        self.models[i].load_state_dict(new_params[i])

In [16]:
class Worker():
  def __init__(self,trainset,valset,testset):
    self.trainloader = torch.utils.data.DataLoader(trainset,batch_size=args.batch_size,shuffle=True,num_workers=2)
    self.valloader = torch.utils.data.DataLoader(valset,batch_size=args.test_batch,shuffle=False,num_workers=2)
    self.testloader = torch.utils.data.DataLoader(testset,batch_size=args.test_batch,shuffle=False,num_workers=2)
    self.models = None
    self.cluster = None
    self.train_data_num = len(trainset)
    self.test_data_num = len(testset)
    self.aggregation_weight = None

  def local_train(self):
    acc_train,loss_train = train(self.models[self.cluster],args.criterion,self.trainloader,args.local_epochs)
    acc_valid,loss_valid = test(self.models[self.cluster],args.criterion,self.valloader)
    return acc_train,loss_train,acc_valid,loss_valid

  def clustering(self,models):
    for i in range(args.cluster_num):
      if i==0:
        cluster = 0
        _,loss = test(models[i],args.criterion,self.trainloader)
      else:
        _,tmp = test(models[i],args.criterion,self.trainloader)
        if tmp<loss:
          cluster = i
    self.cluster = cluster
    

In [17]:
def train(model,criterion,trainloader,epochs):
  optimizer = optim.SGD(model.parameters(),lr=args.lr,momentum=args.momentum,weight_decay=args.weight_decay)
  model.train()
  for epoch in range(epochs):
    running_loss = 0.0
    correct = 0
    count = 0
    for (data,labels) in trainloader:
      data,labels = Variable(data),Variable(labels)
      data,labels = data.to(args.device),labels.to(args.device)
      optimizer.zero_grad()
      outputs = model(data)
      loss = criterion(outputs,labels)
      running_loss += loss.item()
      predicted = torch.argmax(outputs,dim=1)
      correct += (predicted==labels).sum().item()
      count += len(labels)
      loss.backward()
      torch.nn.utils.clip_grad_norm_(model.parameters(), args.clip)
      optimizer.step()

  return 100.0*correct/count,running_loss/len(trainloader)

In [18]:
def test(model,criterion,testloader):
  model.eval()
  running_loss = 0.0
  correct = 0
  count = 0
  for (data,labels) in testloader:
    data,labels = data.to(args.device),labels.to(args.device)
    outputs = model(data)
    running_loss += criterion(outputs,labels).item()
    predicted = torch.argmax(outputs,dim=1)
    correct += (predicted==labels).sum().item()
    count += len(labels)

  accuracy = 100.0*correct/count
  loss = running_loss/len(testloader)


  return accuracy,loss

In [19]:
class Early_Stopping():
  def __init__(self,partience):
    self.step = 0
    self.loss = float('inf')
    self.partience = partience

  def validate(self,loss):
    if self.loss<loss:
      self.step += 1
      if self.step>self.partience:
        return True
    else:
      self.step = 0
      self.loss = loss

    return False

In [20]:
server = Server()
workers = server.create_worker(federated_trainset,federated_valset,federated_testset)
acc_train = []
loss_train = []
acc_valid = []
loss_valid = []

early_stopping = Early_Stopping(args.partience)

start = time.time()

for epoch in range(args.global_epochs):
  sample_worker = server.sample_worker(workers)
  server.send_models(sample_worker)

  acc_train_avg = 0.0
  loss_train_avg = 0.0
  acc_valid_avg = 0.0
  loss_valid_avg = 0.0
  for worker in sample_worker:
    worker.clustering(worker.models)
    acc_train_tmp,loss_train_tmp,acc_valid_tmp,loss_valid_tmp = worker.local_train()
    acc_train_avg += acc_train_tmp/len(sample_worker)
    loss_train_avg += loss_train_tmp/len(sample_worker)
    acc_valid_avg += acc_valid_tmp/len(sample_worker)
    loss_valid_avg += loss_valid_tmp/len(sample_worker)
  server.aggregate_models(sample_worker)
  '''
  for i in range(args.cluster_num):
    server.models[i].to(args.device)
  for worker in workers:
    worker.clustering(server.models)
    acc_valid_tmp,loss_valid_tmp = test(server.models[worker.cluster],args.criterion,worker.valloader)
    acc_valid_avg += acc_valid_tmp/len(workers)
    loss_valid_avg += loss_valid_tmp/len(workers)
  for i in range(args.cluster_num):
    server.models[i].to('cpu')
  '''
  print('Epoch{}  loss:{}  accuracy:{}'.format(epoch+1,loss_valid_avg,acc_valid_avg))
  acc_train.append(acc_train_avg)
  loss_train.append(loss_train_avg)
  acc_valid.append(acc_valid_avg)
  loss_valid.append(loss_valid_avg)

  if early_stopping.validate(loss_valid_avg):
    print('Early Stop')
    break
    
end = time.time()

Epoch1  loss:3.8949894070625306  accuracy:4.464105107995978
Epoch2  loss:3.787770712375641  accuracy:5.020098242243181
Epoch3  loss:3.7427849531173707  accuracy:5.172259488629352
Epoch4  loss:3.7163455724716186  accuracy:5.077942462256575
Epoch5  loss:3.6984432458877574  accuracy:5.306792976821377
Epoch6  loss:3.6842944979667664  accuracy:5.455144625173024
Epoch7  loss:3.6734286427497866  accuracy:5.152580754664903
Epoch8  loss:3.6598856329917906  accuracy:5.11891600671444
Epoch9  loss:3.6575195908546454  accuracy:6.128868939172069
Epoch10  loss:3.6442091584205625  accuracy:5.2178171056155405
Epoch11  loss:3.650455701351165  accuracy:5.522837482064488
Epoch12  loss:3.6430820941925046  accuracy:6.171331844844565
Epoch13  loss:3.6310708284378053  accuracy:6.037164424962859
Epoch14  loss:3.6323269844055175  accuracy:5.8605763982006165
Epoch15  loss:3.6335554122924805  accuracy:5.8969237132935755
Epoch16  loss:3.61643728017807  accuracy:5.731390833474982
Epoch17  loss:3.6191269516944886  a

Epoch136  loss:1.6023478090763088  accuracy:65.38430488390881
Epoch137  loss:1.5715823829174043  accuracy:65.53544983605961
Epoch138  loss:1.5697800159454347  accuracy:65.45749073291398
Epoch139  loss:1.5511331617832185  accuracy:65.98384003994363
Epoch140  loss:1.5640383660793302  accuracy:65.22441582606969
Epoch141  loss:1.5545913696289062  accuracy:65.87604541976948
Epoch142  loss:1.5384962260723116  accuracy:66.5584941424252
Epoch143  loss:1.5340894401073457  accuracy:66.13075896180399
Epoch144  loss:1.5218837797641753  accuracy:66.95257281692668
Epoch145  loss:1.5166381537914277  accuracy:66.2580762079841
Epoch146  loss:1.5122770130634307  accuracy:66.85086575761278
Epoch147  loss:1.4982816427946088  accuracy:66.10176594143906
Epoch148  loss:1.4990354388952256  accuracy:66.70898004718676
Epoch149  loss:1.4929478436708452  accuracy:66.4080315466061
Epoch150  loss:1.4810477912425992  accuracy:66.99560672607694
Epoch151  loss:1.4641760379076005  accuracy:66.60272328158032
Epoch152  l

Epoch270  loss:1.0425945550203322  accuracy:72.94632295012633
Epoch271  loss:1.0378105103969575  accuracy:73.6001781441965
Epoch272  loss:1.0431677758693696  accuracy:73.09865952472742
Epoch273  loss:1.036410677433014  accuracy:73.04876748039918
Epoch274  loss:1.0345722913742066  accuracy:73.42126902646308
Epoch275  loss:1.0239732652902602  accuracy:73.67227828868346
Epoch276  loss:1.0353278309106824  accuracy:73.63252034083546
Epoch277  loss:1.0284751236438752  accuracy:73.70030022478291
Epoch278  loss:1.0389308035373688  accuracy:73.49830877431584
Epoch279  loss:1.0252762943506242  accuracy:73.55235940304371
Epoch280  loss:1.02361426949501  accuracy:73.48355479081404
Epoch281  loss:1.0319464772939682  accuracy:73.79734426810636
Epoch282  loss:1.0223186641931536  accuracy:74.25082522878361
Epoch283  loss:1.0156314611434938  accuracy:73.0377664237617
Epoch284  loss:1.0306550920009612  accuracy:73.21862960984728
Epoch285  loss:1.0245941072702407  accuracy:73.58752688770254
Epoch286  los

In [22]:
acc_test = []
loss_test = []

for i in range(args.cluster_num):
  server.models[i].to(args.device)

nums = 0
for worker in workers:
  nums += worker.test_data_num

start = time.time()

for i,worker in enumerate(workers):
  worker.aggregation_weight = 1.0*worker.test_data_num/nums
  worker.clustering(server.models)
  acc_tmp,loss_tmp = test(server.models[worker.cluster],args.criterion,worker.testloader)
  acc_test.append(acc_tmp)
  loss_test.append(loss_tmp)
  print('Worker{} accuracy:{}  loss:{}'.format(i+1,acc_tmp,loss_tmp))

end = time.time()

acc_test_avg = sum(acc_test)/len(acc_test)
loss_test_avg = sum(loss_test)/len(loss_test)
print('Test  loss:{}  accuracy:{}'.format(loss_test_avg,acc_test_avg))

Worker1 accuracy:64.70588235294117  loss:1.3955684900283813
Worker2 accuracy:85.71428571428571  loss:0.4342481195926666
Worker3 accuracy:77.5  loss:0.8516443371772766
Worker4 accuracy:66.66666666666667  loss:1.7213053703308105
Worker5 accuracy:82.3529411764706  loss:0.5433621406555176
Worker6 accuracy:76.19047619047619  loss:0.6007764339447021
Worker7 accuracy:68.42105263157895  loss:0.9520846009254456
Worker8 accuracy:81.25  loss:0.60696941614151
Worker9 accuracy:57.89473684210526  loss:2.0450634956359863
Worker10 accuracy:82.92682926829268  loss:0.5586618781089783
Worker11 accuracy:72.22222222222223  loss:1.538760781288147
Worker12 accuracy:68.42105263157895  loss:1.3560235500335693
Worker13 accuracy:73.6842105263158  loss:1.2801992893218994
Worker14 accuracy:56.666666666666664  loss:1.9344562292099
Worker15 accuracy:78.94736842105263  loss:0.9946979880332947
Worker16 accuracy:88.23529411764706  loss:0.691483736038208
Worker17 accuracy:66.66666666666667  loss:1.4780722856521606
Worke

In [24]:
acc_tune_test = []
loss_tune_test = []
acc_tune_valid = []
loss_tune_valid = []

start = time.time()

for i,worker in enumerate(workers):
    worker.models = copy.deepcopy(server.models)
    worker.models[worker.cluster] = worker.models[worker.cluster].to(args.device)
    _,_,acc_tmp,loss_tmp = worker.local_train()
    acc_tune_valid.append(acc_tmp)
    loss_tune_valid.append(loss_tmp)
    print('Worker{} Valid accuracy:{}  loss:{}'.format(i+1,acc_tmp,loss_tmp))
    
    acc_tmp,loss_tmp = test(worker.models[worker.cluster],args.criterion,worker.testloader)
    acc_tune_test.append(acc_tmp)
    loss_tune_test.append(loss_tmp)
    print('Worker{} Test accuracy:{}  loss:{}'.format(i+1,acc_tmp,loss_tmp))
    for i in range(args.cluster_num):
        worker.models[i] = worker.models[i].to('cpu')
    del worker.models

end = time.time()

acc_valid_avg = sum(acc_tune_valid)/len(acc_tune_valid)
loss_valid_avg = sum(loss_tune_valid)/len(loss_tune_valid)
print('Validation(tune)  loss:{}  accuracy:{}'.format(loss_valid_avg,acc_valid_avg))
acc_test_avg = sum(acc_tune_test)/len(acc_tune_test)
loss_test_avg = sum(loss_tune_test)/len(loss_tune_test)
print('Test(tune)  loss:{}  accuracy:{}'.format(loss_test_avg,acc_test_avg))

Worker1 Valid accuracy:74.4186046511628  loss:0.6290313601493835
Worker1 Test accuracy:82.3529411764706  loss:1.0059294700622559
Worker2 Valid accuracy:81.91489361702128  loss:0.6034848690032959
Worker2 Test accuracy:88.57142857142857  loss:0.3817615509033203
Worker3 Valid accuracy:68.57142857142857  loss:0.9337775111198425
Worker3 Test accuracy:80.0  loss:0.741433322429657
Worker4 Valid accuracy:69.56521739130434  loss:1.4920322895050049
Worker4 Test accuracy:66.66666666666667  loss:1.7271791696548462
Worker5 Valid accuracy:81.31868131868131  loss:0.6119040250778198
Worker5 Test accuracy:85.29411764705883  loss:0.4559059739112854
Worker6 Valid accuracy:70.9090909090909  loss:0.9951586723327637
Worker6 Test accuracy:71.42857142857143  loss:0.7637213468551636
Worker7 Valid accuracy:80.0  loss:0.7972589731216431
Worker7 Test accuracy:73.6842105263158  loss:0.967278003692627
Worker8 Valid accuracy:72.94117647058823  loss:0.9024269580841064
Worker8 Test accuracy:81.25  loss:0.6584511995315

In [26]:
cluster = [0]*args.cluster_num
for worker in workers:
  cluster[worker.cluster] += 1

print(cluster)

[14, 6]
